# Estimation of Pi Using Random Numbers

Adapted from HPC Carpentries lesson [HPC Parallelisation For Novices](https://www.hpc-carpentry.org/hpc-parallel-novice/).

We will use the Julia programming language to estimate the value of $\pi$.

## The Problem

Consider a square enclosing a quarter of a circle with radius $R$. The area of the square is $A_s$ is <br>
    
$A_s = R^2$

and the area of the quarter circle $A_{qc}$ is
    
$A_{qc} = \frac{1}{4} \pi R^2$.

Substituting for $R^2$, one arrives at

$A_{qc} = \frac{1}{4} \pi A_s$

$\pi = 4 \frac{A_{qc}}{A_s}$

meaning that $\pi$ is equal to four times the ratio of the areas of the quarter circle to the square. Therefore, we can use random numbers to calculate the value of $\pi$ by constructing a unit square, throwing random points into the square, and counting the number of points which land in the quarter circle.

![image](estimate_pi.svg "Title")

The number of points which land in the quarter circle divided by the total number of random points we threw should give us an approximation of the ratio of the areas (with the accuracy increasing with the number of total points used).

We now solve this problem using various methodologies to illustrate the power of vectorization and parallelization.

In [18]:
using IJulia
IJulia.installkernel("Julia 4 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "4",
))

Threads.@threads for i in 1:4
    println(Threads.threadid())
end

1
1
1
1


┌ Info: Installing Julia 4 Threads kernelspec in /Users/tok/Library/Jupyter/kernels/julia-4-threads-1.10
└ @ IJulia /Users/tok/.julia/packages/IJulia/bHdNn/deps/kspec.jl:105


## Using For Loops

The simplest method involves using a single for loop to generate the random points and determine whether the point falls within the circle.

In [1]:
numberOfPoints = 99999999

numberOfCirclePoints = 0

for i in 1:numberOfPoints
    x = rand()
    y = rand()
    r = sqrt(x*x + y*y)
    if r <= 1
        numberOfCirclePoints += 1.0
    end
end

pi = 4 * numberOfCirclePoints / numberOfPoints
print(pi)

3.141427471414275

Let us now put the above code into a function so that we can call it later to do comparisons against other methods.

In [9]:
function serial_pi(numberOfPoints)

    numberOfCirclePoints = 0
    for i in 1:numberOfPoints
        x = rand()
        y = rand()
        r = sqrt(x^2 + y^2)
        if r <= 1
            numberOfCirclePoints += 1
        end
    end
    
    return(4 * numberOfCirclePoints / numberOfPoints)
end

serial_pi (generic function with 1 method)

Now we can call the function with

In [10]:
pi = serial_pi(999999)
print(pi)

3.138135138135138

## Using Threads


In [13]:

import Base.Threads.@spawn

function thread__pi(numberOfPoints)

    numberOfCirclePoints = 0
    Threads.@threads for i in 1:numberOfPoints
        x = rand()
        y = rand()
        r = sqrt(x^2 + y^2)
        if r <= 1
            numberOfCirclePoints += 1
        end
    end
    
    return(4 * numberOfCirclePoints / numberOfPoints)
end

thread__pi (generic function with 1 method)

In [16]:
using BenchmarkTools

@btime serial_pi(12)

  24.054 ns (0 allocations: 0 bytes)


3.3333333333333335

In [17]:
@btime thread_pi(12)

  3.333 μs (10 allocations: 960 bytes)


3.0